# FIFS

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import sys, os
import pandas as pd
import matplotlib.pyplot as plt

from csidataset import *
import data_loader
from data_loader import *
sys.path.append("/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool")
import denoise

In [2]:
base_path = "/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/csi_dataset/localization_phone/1223_phone/5G/20MHz/csv/all"

In [14]:
reference_points = {}

spacing = 0.6  # 每隔 0.6m

for ref_id, coord in data_loader.COORDINATES.items():
    folder_path = os.path.join(base_path, f"reference_point_{ref_id}.xlsx")
    reference_points[folder_path] = (ref_id, coord)

In [15]:
data, rp_labels, coord_labels = load_data(reference_points)
amp_data = np.array(data.iloc[:, :48])  
phase_data = np.array(data.iloc[:, 48:-2]) 

In [16]:
encoder = OneHotEncoder(sparse_output=False)
one_hot_labels = encoder.fit_transform(np.array(rp_labels).reshape(-1, 1))

In [17]:
amp_train, amp_temp, y_train, y_temp = train_test_split(amp_data, one_hot_labels, test_size=0.3, random_state=42)
amp_val, amp_test, y_val, y_test = train_test_split(amp_temp, y_temp, test_size=1/3, random_state=42)

In [18]:
import numpy as np

def group_average(amp, num_groups=4):
    # amp: (N_samples, N_subcarriers)
    n, d = amp.shape
    group_size = d // num_groups
    features = []
    for i in range(num_groups):
        s = i * group_size
        e = (i+1) * group_size if i < num_groups-1 else d
        features.append(amp[:, s:e].mean(axis=1, keepdims=True))
    return np.concatenate(features, axis=1)  # shape: (N, num_groups)


In [19]:
def build_radio_map(amp_train, y_train, num_groups=4):
    rp_train = np.argmax(y_train, axis=1)  # (N_train,)
    num_rp = y_train.shape[1]
    amp_feat = group_average(amp_train, num_groups=num_groups)  # (N_train, num_groups)
    radio_map = np.zeros((num_rp, amp_feat.shape[1]))
    for i in range(num_rp):
        idx = (rp_train == i)
        radio_map[i] = amp_feat[idx].mean(axis=0)
    return radio_map


In [20]:
def pearson_corr(x, y):
    vx = x - np.mean(x)
    vy = y - np.mean(y)
    denom = np.sqrt(np.sum(vx**2)) * np.sqrt(np.sum(vy**2))
    if denom == 0:
        return 0
    return np.sum(vx * vy) / denom

def fifs_predict(amp_eval, radio_map, num_groups=4):
    amp_feat = group_average(amp_eval, num_groups=num_groups)  # (N_eval, num_groups)
    N = amp_feat.shape[0]
    num_rp = radio_map.shape[0]
    preds = np.zeros(N, dtype=int)
    for i in range(N):
        scores = np.array([pearson_corr(amp_feat[i], radio_map[j]) for j in range(num_rp)])
        preds[i] = np.argmax(scores)
    return preds


In [21]:
def mean_distance_error(preds, labels, rp_coords):
    coords_pred = np.array([rp_coords[i] for i in preds])
    coords_true = np.array([rp_coords[i] for i in labels])
    dist = np.linalg.norm(coords_pred - coords_true, axis=1)
    return np.mean(dist)


In [ ]:
COORDINATES = {
    # 下邊界 (1-10 和 40-31)
    1: (0, 0), 40: (0.6, 0), 39: (1.2, 0), 38: (1.8, 0), 37: (2.4, 0),
    36: (3.0, 0), 35: (3.6, 0), 34: (4.2, 0), 33: (4.8, 0), 32: (5.4, 0), 31: (6.0, 0),

    # 左邊界 (1-11)
    2: (0, 0.6), 3: (0, 1.2), 4: (0, 1.8), 5: (0, 2.4),
    6: (0, 3.0), 7: (0, 3.6), 8: (0, 4.2), 9: (0, 4.8), 10: (0, 5.4), 11: (0, 6.0),

    # 上邊界 (11-21)
    12: (0.6, 6.0), 13: (1.2, 6.0), 14: (1.8, 6.0), 15: (2.4, 6.0),
    16: (3.0, 6.0), 17: (3.6, 6.0), 18: (4.2, 6.0), 19: (4.8, 6.0),
    20: (5.4, 6.0), 21: (6.0, 6.0),

    # 右邊界 (21-31)
    22: (6.0, 5.4), 23: (6.0, 4.8), 24: (6.0, 4.2), 25: (6.0, 3.6),
    26: (6.0, 3.0), 27: (6.0, 2.4), 28: (6.0, 1.8), 29: (6.0, 1.2), 30: (6.0, 0.6),

    # 中間點 (41-49)
    41: (3.0, 0.6), 42: (3.0, 1.2), 43: (3.0, 1.8),
    44: (3.0, 2.4), 45: (3.0, 3.0), 46: (3.0, 3.6),
    47: (3.0, 4.2), 48: (3.0, 4.8), 49: (3.0, 5.4)
}

In [29]:
rp_coords = [COORDINATES[i+1] for i in range(49)]


# ==== 前處理分組數 ====
num_groups = 8  # 按FIFS原論文，4~8組常見，可調整

# ==== Radio Map 建立 ====
radio_map = build_radio_map(amp_train, y_train, num_groups=num_groups)

# ==== 預測 ====
gt_val = np.argmax(y_val, axis=1)
gt_test = np.argmax(y_test, axis=1)

preds_val = fifs_predict(amp_val, radio_map, num_groups=num_groups)
preds_test = fifs_predict(amp_test, radio_map, num_groups=num_groups)

# ==== 評估 ====
acc_val = np.mean(preds_val == gt_val)
acc_test = np.mean(preds_test == gt_test)
print(f'Val ACC: {acc_val*100:.2f}%')
print(f'Test ACC: {acc_test*100:.2f}%')

# 如果你有rp_coords（如: [(0,0),(0,0.6)...]）
mde_val = mean_distance_error(preds_val, gt_val, rp_coords)
mde_test = mean_distance_error(preds_test, gt_test, rp_coords)
print(f'Val MDE: {mde_val:.3f} m')
print(f'Test MDE: {mde_test:.3f} m')


Val ACC: 51.20%
Test ACC: 52.04%
Val MDE: 2.035 m
Test MDE: 1.911 m
